# NYC 311 Service Requests Data Pipeline

*Library + jobs to fetch, cache, and analyze NYC 311 service request data.*

# As Jobs

## LTP Pipeline

In [ ]:
from pathlib import Path

# Per-user, cross-platform base under the home dir
# retrieve for one day only:
dir_base = Path.home() / "TASK_HBC_TSY"

%cd -q ..
!python -m hbc.jobs.dispatch \
  --job-name=job_fetch_nyc_open_data_311_service_requests \
  --as-of=20091231 \
  --dir-base={dir_base} \
  --incremental=True \
  --log-level=INFO
%cd -q notebooks/

In [ ]:
# restore cache integrity for the last missing dates
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_fetch_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --incremental=false \
      --log-level=INFO \
      --last-missing-dates=5
%cd -q notebooks/

## Analytics

In [ ]:
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_analyse_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --log-level=INFO \
      --n-worst=10 \
      --n-best=10 \
      --n-days=10
%cd -q notebooks/

***

# As Library 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Imports

In [2]:
import sys
from pathlib import Path

p = str(Path.cwd().parent) # one dir up
if p not in sys.path:
    sys.path.insert(0, p)

import pandas as pd
import os
import numpy as np
import logging
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Api

In [5]:
from hbc import app_context, DataContainer, utils as ul

from hbc.quant.analysis import AnalyticalEngine
from hbc.quant.plots import PlotEngine

In [6]:
app_context

AppContext
as_of : 2025-12-25
dir_base: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp'),
dir_analytics: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/ANALYTICS'),
dir_logging: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS')

## Logging

In [7]:
import logging
logger = logging.getLogger()

In [9]:
# Console-only (no file writes):
ul.conf_log(level=logging.DEBUG, console=True, file=False, reset_handlers=True)

# File-only (no console output at all):
ul.conf_log(level=logging.INFO, console=False, file=True, reset_handlers=True)

# Both:
ul.conf_log(level=logging.INFO, console=True, file=True, reset_handlers=True)

Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt
Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt


<RootLogger root (INFO)>

# Cache To SQLLite DataBase Via Asp.Net Rest Api with EF Core

In [10]:
import hbc
from hbc import DataContainer
from hbc.ltp.persistence.db import SqlLiteDataBase

In [11]:
logger.setLevel(logging.INFO)

In [12]:
# reset database
db = SqlLiteDataBase()
db.all_dbs
db.all_tables
db.execute(f'delete from nyc_open_data_311_customer_satisfaction_survey')

4889

In [15]:
import os
from pathlib import Path

dc = DataContainer("nyc_open_data_311_customer_satisfaction_survey")
# retrieve: query first 100 rows
dc.get()
dc.df.shape

2025-12-25 11:11:59 fetch_nycopen.py       72 INFO  root    : Fetched 100 rows
2025-12-25 11:11:59 base.py                52 INFO  root    : using validator: ValidatorGeneric
2025-12-25 11:11:59 base.py                53 INFO  root    : cleaning...
2025-12-25 11:11:59 base.py                56 INFO  root    : normalizing...
2025-12-25 11:11:59 base.py                59 INFO  root    : validating...
2025-12-25 11:11:59 base.py                62 INFO  root    : finalizing...


(100, 14)

In [16]:
# retrieve: query distinct
dc.get(query="$apply=groupby((campaign))")
dc.df.shape

2025-12-25 11:12:06 fetch_nycopen.py       43 INFO  root    : using pagination at fetching with page_size=10000 timeout=30
2025-12-25 11:12:06 fetch_nycopen.py       72 INFO  root    : Fetched 6 rows
2025-12-25 11:12:06 base.py                52 INFO  root    : using validator: ValidatorGeneric
2025-12-25 11:12:06 base.py                53 INFO  root    : cleaning...
2025-12-25 11:12:06 base.py                56 INFO  root    : normalizing...
2025-12-25 11:12:06 base.py                59 INFO  root    : validating...
2025-12-25 11:12:06 base.py                62 INFO  root    : finalizing...
2025-12-25 11:12:06 container.py           63 ERROR root    : DataContainer nyc_open_data_311_customer_satisfaction_survey does not adhere to schema. Missing columns: agent_customer_service, agent_job_knowledge, answer_satisfaction, channel, completion_time, nps, overall_satisfaction, start_time, survey_language, survey_type, wait_time, year


(6, 2)

In [17]:
# retrieve: query with filter
dc.get(query="$filter=campaign eq 'Campaign 4'")
dc.df.shape

2025-12-25 11:12:11 fetch_nycopen.py       43 INFO  root    : using pagination at fetching with page_size=10000 timeout=30
2025-12-25 11:12:12 fetch_nycopen.py       72 INFO  root    : Fetched 4889 rows
2025-12-25 11:12:12 base.py                52 INFO  root    : using validator: ValidatorGeneric
2025-12-25 11:12:12 base.py                53 INFO  root    : cleaning...
2025-12-25 11:12:12 base.py                56 INFO  root    : normalizing...
2025-12-25 11:12:12 base.py                59 INFO  root    : validating...
2025-12-25 11:12:12 base.py                62 INFO  root    : finalizing...


(4889, 14)

In [19]:
# caching: 
dc.to_cache()

2025-12-25 11:13:03 rest.py               140 INFO  root    : Synced 4889 rows via batch API


In [20]:
# cahcing: get 10 rows
dc.from_cache().shape

(10, 15)

In [21]:
# caching: get by filter
dc.from_cache(query="$filter=campaign eq 'Campaign 4'").shape

(4889, 15)

In [22]:
# caching: get distinct
dc.from_cache(query="$apply=groupby((year))").shape

(4, 1)

In [23]:
# caching: get page 2 with page size 50
dc.from_cache(query="$top=50&$skip=50").shape

(50, 15)

In [24]:
# caching: get total count
dc.from_cache(query="$count=true").shape

(4889, 15)

## DataContainer

In [25]:
dc = DataContainer('nyc_open_data_311_service_requests')

In [26]:
# we retrieve 100 rows
dc.get()

2025-12-25 11:13:21 fetch_nycopen.py       72 INFO  root    : Fetched 100 rows
2025-12-25 11:13:21 base.py                52 INFO  root    : using validator: ValidatorNYCOpen311Service
2025-12-25 11:13:21 base.py                53 INFO  root    : cleaning...
2025-12-25 11:13:21 base.py                56 INFO  root    : normalizing...
2025-12-25 11:13:21 base.py                59 INFO  root    : validating...
2025-12-25 11:13:21 valid_nycopen.py      183 INFO  root    : Validation summary -> flagged 10 rows. closed_date before created_date: 4; closed_date set but status not closed: 5; incident_zip outside NYC range: 2; resolution_action_updated_date before created_date: 7
2025-12-25 11:13:21 base.py                62 INFO  root    : finalizing...
2025-12-25 11:13:21 container.py           63 ERROR root    : DataContainer nyc_open_data_311_service_requests does not adhere to schema. Missing columns: bridge_highway_direction, bridge_highway_name, bridge_highway_segment, garage_lot_name, l

In [29]:
app_context.as_of = ul.str_as_date('20091231')

In [30]:
dc.get(query=f"$filter=created_date eq '{ul.date_as_iso_format(app_context.as_of)}'")

2025-12-25 11:14:39 fetch_nycopen.py       43 INFO  root    : using pagination at fetching with page_size=10000 timeout=30
2025-12-25 11:14:40 fetch_nycopen.py       72 INFO  root    : Fetched 4155 rows
2025-12-25 11:14:40 base.py                52 INFO  root    : using validator: ValidatorNYCOpen311Service
2025-12-25 11:14:40 base.py                53 INFO  root    : cleaning...
2025-12-25 11:14:40 base.py                56 INFO  root    : normalizing...
2025-12-25 11:14:40 base.py                59 INFO  root    : validating...
2025-12-25 11:14:40 valid_nycopen.py      183 INFO  root    : Validation summary -> flagged 311 rows. closed_date before created_date: 52; closed_date set but status not closed: 89; incident_zip outside NYC range: 4; resolution_action_updated_date before created_date: 270
2025-12-25 11:14:40 base.py                62 INFO  root    : finalizing...


In [31]:
dc.to_cache()

2025-12-25 11:14:44 rest.py               132 ERROR root    : Batch POST /nyc_open_data_311_service_requests/batch failed with status 404: 


HTTPError: 404 Client Error: Not Found for url: http://localhost:5047/nyc_open_data_311_service_requests/batch

In [ ]:
dc.from_cache().head()

## Analytics

In [ ]:
df = dc.df

In [ ]:
cols = ul.cols_as_named_tuple(df)

In [ ]:
df = df[~df[cols.DROP_FLAG]]

In [ ]:
df["hbc_days_to_close"] = (
    pd.to_datetime(df[cols.closed_date])
    - pd.to_datetime(df[cols.created_date])
).dt.days.astype("Int64")
cols = ul.cols_as_named_tuple(df)

In [ ]:
m = df[cols.hbc_days_to_close] == 0
df_closed_not_same_day = df[~m]

In [ ]:
path = ul.path_to_str(
                ul.mk_dir(ul.get_dir_analytics() / "plots")
                / "closed_requests_by_location.html"
            )
_ = PlotEngine.plot_geo_map(
            df=df_closed_not_same_day,
            col_latitude=cols.latitude,
            col_longitude=cols.longitude,
            aggregation="count",
            round_precision=3,
            cluster=True,
            start_zoom=11,
            tiles="CartoDB positron",
            savepath= path
        )
print(path)

In [ ]:
# by agency
res = AnalyticalEngine.descriptive_stats(
    n_best=10,
    n_worst=10,
    df=df_closed_not_same_day,
    col_metric=cols.hbc_days_to_close,
    group=[
        cols.agency,
        cols.agency_name,
    ],
)


In [ ]:
res.keys()

In [ ]:
res['worst']

### new dataset: nyc_open_data_311_call_center_inquiry:

In [ ]:
dc = DataContainer('nyc_open_data_311_call_center_inquiry')

In [ ]:
dc.get()

In [ ]:
dc.get(where=f"agency='NYPD'", limit=250)

In [ ]:
dc.df.shape

In [ ]:
dc.df.head()

In [ ]:
dc.df.head()

In [ ]:
dc.get(where=f"date = '{ul.date_as_iso_format(ul.str_as_date('2010-01-03'))}'")

In [ ]:
dc.df.head()

### new_dataset: nyc_open_data_311_customer_satisfaction_survey

In [ ]:
dc = DataContainer('nyc_open_data_311_customer_satisfaction_survey')

In [ ]:
dc.get()

In [ ]:
dc.get(where=f"answer_satisfaction='Neutral'")

In [ ]:
dc.df.head()